# Designing networks: The Creator class

## MagicLocations

Locations make the life of an agent-based modeler a lot easier.
However, it is still very tidious to connect a large population of agents via a specific network structure.
Therefore, popy provides a work flow which ....


### Overview

In [1]:
import popy

class MyLocation(popy.MagicLocation):
    n_agents: int | None = None
    overcrowding: bool | None = None
    only_exact_n_agents: bool = False
    n_locations: int | None = None
    static_weight: bool = False
    recycle: bool = True

    def filter(self, agent: popy.Agent) -> bool:
        """Assigns only agents with certain attributes to this location."""
        return True

    def split(self, agent: popy.Agent) -> str | float | str | list | None:
        """Creates seperated location instances for each value of an agent-attribute."""
        return None

    def stick_together(self, agent: popy.Agent) -> str | float:
        """Ensures that agents with a shared value on an attribute are assigned to the same
        location instance."""
        return agent.id

    def weight(self, agent: popy.Agent) -> float:
        """Defines the edge weight between the agent and the location instance."""
        return 1

    def project_weights(self, agent1: popy.Agent, agent2: popy.Agent) -> float:
        """Calculates the edge weight between two agents that are assigned to the same location 
        instance."""
        return min([self.get_weight(agent1), self.get_weight(agent2)])

    def find(self, agent: popy.Agent) -> bool:
        """Assigns the agent to a specific location instance that meets the requirements."""
        return True

    def nest(self) -> popy.Location | None:
        """Ensures that the agents assigned to the same instance of this location class
        are also assigned to the same instance of the returned location class."""
        return None

    def melt(self) -> list[popy.Location] | tuple[popy.Location]:
        """Merges the agents assigned to the instances of the returned location classes
        into one instance of this location class."""
        return []

## Generating static networks

Let's start with creating a static network.

### Generating Agents

Although the main task of Popy is connecting agents via the definition of locations, Popy can also generate agents for you based on micro-level data.
This makes it easy to create *empirical-based* agents from, for instance, survey data.
For the following example, we assume that we want to create a network model for schools and have access to a sample of individual data collected in a school.

This is our example data:

In [2]:
import pandas as pd

df_school = pd.read_csv("example_school_data.csv", sep=";")
df_school.head()

,status,gender,grade,hours,friend_group
0,pupil,m,1.0,4,1
1,pupil,w,1.0,4,1
2,pupil,m,1.0,4,2
3,pupil,w,1.0,4,2
4,pupil,m,2.0,4,3


In [3]:
df_school.tail()

,status,gender,grade,hours,friend_group
37,teacher,w,NaN,9,0
38,teacher,m,NaN,10,0
39,teacher,w,NaN,5,0
40,social_worker,w,NaN,6,0
41,social_worker,w,NaN,4,0


The process of generating agents (and locations) is handled by the `Creator`.
Hence, the first step after creating a `Model` is the creation of an instance of `Creator`.

In [4]:
model = popy.Model()
creator = popy.Creator(model)

To create agents from this dataset, we can simply use the method `create_agents()`.
This method creates one agent object for each row in the given dataset.
Each column is translated into an agent attribute with the corresponding value.

In [5]:
agents = creator.create_agents(df=df_school)
agents

AgentList (42 objects)

The method `create_agents()` generates an [AgentPy-AgentList](https://agentpy.readthedocs.io/en/latest/reference_sequences.html#containers) containing the instances of `popy.Agent`.
Let's have look at the first agent:

In [6]:
type(agents[0])

popy.agent.Agent

In [7]:
vars(agents[0])

{'_var_ignore': [],
 'id': 1,
 'type': 'Agent',
 'log': {},
 'model': Model,
 'p': {},
 'status': 'pupil',
 'gender': 'm',
 'grade': 1.0,
 'hours': 4,
 'friend_group': 1}

If we do not insert an agent class via the parameter `agent_class`, the default agent class of Popy is used to create the agent instances, but we could also use our own agent class (that inherits from `popy.Agent`):

In [8]:
class MyAgent(popy.Agent):
    pass

model = popy.Model()
creator = popy.Creator(model)
agents = creator.create_agents(
    df=df_school, 
    agent_class=MyAgent,
)
agents

AgentList (42 objects)

In [9]:
type(agents[0])

__main__.MyAgent

For now, this is enough information on how to create agents based on micro-level data using Popy.
More useful options for the generation of agents based on micro-level data can be found in chapter *From survey participants to agents*.

### Generating locations

Now let's turn to the generation of networks.
To generate a network with Popy, one has to define at least one `MagicLocation`.
Those location classes are used to generate location instances which are then connected to the agent instances in order to build a bipartite network.
We start by just using the default class of `MagicLocation` without defining any further attributes or methods.
Every location class defined by the user must inherit - directly or indirectly - from `popy.MagicLocation`.
We call our first location `ClassRoom`.

In [10]:
# Define a Location class
class ClassRoom(popy.MagicLocation):
    pass

In [11]:
import popy.utils as utils
utils._get_cls_as_str(ClassRoom)

'ClassRoom'

Now we use the `Creator`-method `create_locations()` to create the location instances and - and this is important - to assign the agents to the location instances:

In [12]:
creator.create_locations(location_classes=[ClassRoom])
model.locations

LocationList (1 object)

In [13]:
vars(model.agents[0])

{'_var_ignore': [],
 'id': 1,
 'type': 'MyAgent',
 'log': {},
 'model': Model,
 'p': {},
 'status': 'pupil',
 'gender': 'm',
 'grade': 1.0,
 'hours': 4,
 'friend_group': 1,
 'ClassRoom': 'gv=None-None,gid=0'}

In [14]:
type(model.agents[0]).__name__

'MyAgent'

In [15]:
model.locations[0].type

'ClassRoom'

In [16]:
type(model.agents[0])

__main__.MyAgent

For now, we can ignore the location instances as well as the agent instances, as this part of the introduction focuses on the generation of static networks.
The important thing is that `create_locations()` has not only created the location instances with respect to the given agent population, but has also assigned the agents to the location according to the rules specified by the location classes.
We can check this by looking at the bipartite network:

In [17]:
inspector = popy.NetworkInspector(model)
inspector.plot_bipartite_network()

The node in the center of the graph is the location.
All other nodes are agents.
This means that if we use the default location class without further customization, only one location instance is created to which all agents are connected.
Since each agent is assigned to this one location instance, the result is a fully connected agent graph:

In [18]:
inspector.plot_agent_network()

### Generate agents and locations in one step

Before diving into all the details of the definition of location classes, let's simplify the process of generating agents and locations.
The method `make()` combines `create_agents()` and `create_locations()` into one simple method.
However, note that `make()` always creates the agents based on a given dataset.
If you already have a population of agents, use `create_locations()` instead. 

In [19]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

# Let the Creator create agents and locations
creator.make(
    df=df_school,
    location_classes=[ClassRoom],
)

(AgentList (42 objects), LocationList (1 object))

In [20]:
inspector.plot_bipartite_network()

In [21]:
inspector.plot_agent_network()

### Setting the location size

In the next step, we specify the number of people in one classroom.
In this example, we assume tiny classrooms of four agents.
To set the number of agents per location, we need to set the class attribute `n_agents` to the desired value.

In [22]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4

In [23]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (10 objects))

In [24]:
inspector.plot_bipartite_network()

In [25]:
inspector.plot_agent_network()

The network diagrams above now show multiple clusters.
Each cluster represents one classroom.
If we set a specific size for a location, the Creator creates as many location instances with that size as needed.
The agents are then assigned randomly to one of these location instances.
As you can see, some classrooms have less than four members because the number of agents assigned to classrooms cannot be divided exactly by the desired number of four.

The `overcrowding` attribute determines how the number of required locations is rounded.
By default, `overcrowding` is set to `None` which means that the number of required locations is either rounded up or rounded down using `round`.
Below we change `overcrowding` to `True` to create one less location instance and *overcrowd* the classrooms instead.

In [26]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    overcrowding = True

In [27]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (10 objects))

In [28]:
inspector.plot_bipartite_network()

The plot above now shows that there is one class room less and two class rooms have 5 members.
We could also set `round_function` to `round` to round down or up.

If we do not want locations that are either overcrowded or undercrowded but only locations that have exactly the size we defined, we could use the attribute `exact_size_only` to `True`:

In [29]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    exact_size_only = True

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network()

As a consequence, two agents are not assigned to any location.

### Defining the number of locations

The attribute `n_agents` implicitly changes the number of the created locations.
Using the attribute `n_locations`, you can also set the number of locations explicitly:

In [30]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    n_locations = 4

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network()

While in the example above four classrooms with the defined size of 4 are created, in the example below no size is set explicitly:

In [31]:
class ClassRoom(popy.MagicLocation):
    n_locations = 4

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network(node_color="black")

### Specifying location visitors

The classrooms above are made of all agents.
But in many cases we want specific locations to be exclusively accessible to certain agents.
For this scenario the method `filter()` exists.
If this method returns `True`, an agent gets assigned to an instance of this location class.
The most common way to use this method is to specify a condition that requires a certain agent attribute to contain a certain value.

In this example we want classrooms to be only accessible for pupils.

In [32]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"

In [33]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [34]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

Now classrooms consist only of pupils, while all other agents do not belong to any location.

By the way: Besides looking at the network graph, the function `inspector.location_information()` and `inspector.location_crosstab()` provide usefull overviews of the created location instances and the assigned agents:

In [35]:
inspector.location_information(
    select_locations=ClassRoom,
    agent_attributes=["grade", "status"],
    output_format="df",
)

,location_id,grade,status,location_type
0,0,2.0,pupil,ClassRoom
1,0,4.0,pupil,ClassRoom
2,0,1.0,pupil,ClassRoom
3,0,1.0,pupil,ClassRoom
4,1,3.0,pupil,ClassRoom
5,1,2.0,pupil,ClassRoom
6,1,3.0,pupil,ClassRoom
7,1,2.0,pupil,ClassRoom
8,2,2.0,pupil,ClassRoom
9,2,1.0,pupil,ClassRoom


In [36]:
inspector.location_crosstab(
    select_locations=ClassRoom, 
    agent_attributes=["status"],
    output_format="df",
)

[index  location_id status  count location_type
 0                0  pupil      4     ClassRoom
 1                1  pupil      4     ClassRoom
 2                2  pupil      4     ClassRoom
 3                3  pupil      4     ClassRoom
 4                4  pupil      4     ClassRoom
 5                5  pupil      4     ClassRoom
 6                6  pupil      4     ClassRoom
 7                7  pupil      4     ClassRoom]

### Building separated locations

The above table shows that the classrooms are not separated by grade.
To seperate agents by grade, we could define one location class for each grade and use `filter()` to assign only agents with a specific grade value to a specific location.

A more convenient way to do it is to use the method `split()`.
For each agent, the method `split()` returns one value.
For each unique value, seperated location instances are created.
In this case, the method `split()` returns the attribute `grade` for each agents.
Thus, the Creator builds seperate classroom instances for each unique value of the agent attribute `grade`.

In [37]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade

In [38]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="grade")

If we use `split()` in combination with `n_locations`, for each subgroup the desired number of locations gets realized:

In [39]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    n_locations = 1
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade

In [40]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="grade")

### Keeping agents together

In the following plot the nodes are colored by their attribute `friend_group`.
It shows that the members of friend groups are distributed over different classrooms.

In [41]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="friend_group")

Although this is a very realistic situation, in this example, we want that all friend group members are always in the same class.
To implement that, we use the location method `stick_together()`:
For each agent, the method `stick_together()` returns a specific value.
Agents with the same return value are sticked together.

In [42]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

In [43]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="friend_group")

### Edge weights

Until now, all edges between nodes have a weight of `1`.
The location method `weight()` can be used to set different weights.
In the following, we set the weight of all edges generated by a classroom to `5`.
This number could, for instance, represent that agents are together in classrooms for five hours.

In [44]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return 5

In [45]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

agents, locations = creator.make(df=df_school, location_classes=[ClassRoom])

In [46]:
inspector.plot_bipartite_network()

In [47]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="grade")

To implement individual weights between an agent and a location, we could also let `weight()` return an agent attribute. 
In this case we use the agent attribute `agent.hours`:

In [48]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [49]:
inspector.plot_bipartite_network()

In [50]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="grade")

As the value returned by `location.weight()` refers to the weight between the agent and the location, all the weights between the agents and the locations must be somehow combined when determining the weight between agents (aka graph projection).
The location method `project_weights()` defines how those weights are combined.
By default, `project_weights()` returns the smallest weight of the two to be combined.
The code cell below shows how `project_weights()` combines the two weights by default.
In this example, we keep this way of combining the weights, but this method could be easily rewritten.

In [51]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

Note that we use the method `location.get_weight()` to access the weight between the agent and the location.

In [52]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="grade")

### Bringing together different agents

So far, we are able to connect agents who share a certain attribut value.
But what if we want to explicitly connect agents who have different values on a certain attribute?
One solution could be to simply give those agents we want to be in the same location the same value on a certain attribute and then define a location class that brings together these agents.

Another possibility is to *melt* different locations into one location.
To do this, we have to define at least three location classes:  Two or more locations that are the components that get melted into one location and one location that melts those components together.
Those locations that 

Assume we want to create classrooms that consist of one teacher and four pupils.
To create such a location, we first define a `MeltLocation` (`TeachersInClassRoom`) that consists of only one teacher.
Then we define a `MeltLocation` (`PupilsInClassRoom`) that consists of four pupils.
Finally, we define a location (`ClassRoom`) that uses the method `melt()` to melt the two previously defined locations into one location.
The method `melt()` must return a tuple or list of at least two location classes that shall be melted into one.

[Hier hinzufügen, was alles von der Unterklasse und was von der Oberklasse übernommen wird]

In [53]:
# a location for teachers
class TeachersInClassRoom(popy.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

# a location for pupils
class PupilsInClassRoom(popy.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

# a location for teachers and pupils
class ClassRoom(popy.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom

In [54]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

Now we bring back all other settings we made so far:

In [55]:
class TeachersInClassRoom(popy.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

In [56]:
class TeachersInClassRoom(popy.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        statuses = [agent1.status, agent2.status]
        if "pupil" in statuses and "teacher" in statuses:
            return 1 # min([self.get_weight(agent1), self.get_weight(agent2)])
        else:
            return 0
    
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom])
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

### More than one location

The melting of locations combines different locations into one location, but does not create multiple different location classes.
If we want to generate multiple different location types, we have to simply feed more than one location class into the `Creator`.

In the following, we introduce a `School` as a second type of location.
Each school has about `20` members.
(In order to keep the code clean, we skip the melting of locations temporarely.)

In [57]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(popy.MagicLocation):
    n_locations = 2
        

model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom, School])
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

In [58]:
inspector.plot_bipartite_network()

In [59]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

### Nesting locations
The plot above shows two big clusters.
Each of those clusters represents one school.
The plot above also shows something unrealistic: The schools are connected because members of one class are not always in the same school.
We can use the `School`-method `stick_together()` to fix this issue.
This works because whenever an agent is assigned to a location instance, the agent gets a new attribute named after the location class.
This new attribute is set to a location instance identifier value.

In [60]:
class ClassRoom(popy.MagicLocation):
    n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.MagicLocation):
    n_locations = 2

    def stick_together(self, agent):
        return agent.ClassRoom


model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom, School])

(AgentList (42 objects), LocationList (10 objects))

In [61]:
inspector.plot_bipartite_network()

In [62]:
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

Note that it is very important that the agents get assigned to classrooms before getting assigned to schools.
This means that the order of the creation of the locations is important and, hence, the order of the location classes given to the `location_classes` argument.

If we build schools before classrooms, the above method does not work the way intended:

In [63]:
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[School, ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

In [64]:
inspector.plot_bipartite_network()

As we saw, the method `stick_together()` can be used to nest locations into other locations.
However, this approach is limited because we can only specify one location class as the return value of `stick_together()`.

Another way to nest locations into other locations is to use the location method `nest()`.
The method `nest()` can return `None` or a location class.
If `nest()` returns a location class, the location is nested into the returned location class.
For instance, to nest classrooms within schools, we must define the method `nest()` for the location `ClassRoom` and let this method return `School`.
Again, the order of location creation plays a crucial role: The level-1 location must always be created after the level-2 location.

In [65]:
class ClassRoom(popy.MagicLocation):
    def setup(self):
        self.n_agents = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.MagicLocation):
    n_locations = 2


model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[School, ClassRoom])

inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

In [66]:
inspector.plot_bipartite_network()

`nest()` allows us to nest as many locations in as many levels as we want. [Anwendungsbeispiel?]
However, `nest()` has one disadvantage: Because the agents are first grouped into the level-2 location and then into the level-1 location, specific compositions defined at level 1 are not considered when grouping the agents into the level-2 locations.

The following example demonstrates that:

In [67]:
class TeachersInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
class ClassRoom(popy.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.MagicLocation):
    n_locations = 2
    
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[School, ClassRoom])
inspector.plot_bipartite_network()

In [68]:
inspector.location_crosstab(
    select_locations=[ClassRoom], 
    agent_attributes="status",
    output_format="df",
)

[index  location_id   status  count location_type
 0                0    pupil      4     ClassRoom
 1                0  teacher      1     ClassRoom
 2                1    pupil      2     ClassRoom
 3                1  teacher      1     ClassRoom
 4                2    pupil      5     ClassRoom
 5                2  teacher      1     ClassRoom
 6                3    pupil      3     ClassRoom
 7                3  teacher      1     ClassRoom
 8                4    pupil      3     ClassRoom
 9                4  teacher      1     ClassRoom
 10               5    pupil      5     ClassRoom
 11               5  teacher      1     ClassRoom
 12               6    pupil      5     ClassRoom
 13               6  teacher      1     ClassRoom
 14               7    pupil      3     ClassRoom
 15               7  teacher      1     ClassRoom
 16               8    pupil      2     ClassRoom
 17               8  teacher      1     ClassRoom
 18               9    pupil      5     ClassRoom


As you can see in the graph and the table above, the compositions defined by `ClassRoom` are not always met.
That is due to the fact that the agents are first assigned to the schools independently of the settings defined by `ClassRoom`.
When the classrooms are built, there are not always the *necessary* agents in each school needed to meet the composition defined in `ClassRoom`.
This might not always be a problem.
However, if we want to ensure that classrooms always have the defined composition of agents, they have to be created before the schools are created and, thus, we have to use the method `stick_together()` to nest classrooms into schools:

In [69]:
class TeachersInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.MagicLocation):
    n_locations = 2
    
    def stick_together(self, agent):
        return agent.ClassRoom
    
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(df=df_school, location_classes=[ClassRoom, School])
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

It is also possible to combine both nesting techniques.
We could first nest the class `ClassRoom` into `School` using `stick_together()` in order to get the composition wanted for `ClassRoom` and then use `nest()` to nest further locations into `School`, as it is done in the next example:

In [70]:
class TeachersInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.MagicLocation):
    n_locations = 2
    
    def stick_together(self, agent):
        return agent.ClassRoom

class SoccerTeam(popy.MagicLocation):
    n_agents = 11
    
    def nest(self):
        return School
    
model = popy.Model()
creator = popy.Creator(model)
inspector = popy.NetworkInspector(model)

creator.make(
    df=df_school, 
    location_classes=[
        ClassRoom, # nested into School using `stick_together()`
        School, 
        SoccerTeam, # nested into School using `nest()`
        ],
    )
inspector.plot_agent_network(node_attrs=df_school.columns, node_color="status")

In [71]:
inspector.plot_bipartite_network()

This way we can nest multiple locations into one level-2 location and at the same time ensure that the composition of agents defined in `ClassRoom` is met in each school.
The order of the locations plays a crucial role:
1. The classrooms with the desired compositions are created
2. The schools are created keeping together whole classrooms
3. Soccer teams are created within the schools